## 5 minute bar - long tail - ananylics

In [23]:
import warnings
warnings.filterwarnings('ignore')

import time
from datetime import date, datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as ta
import numpy as np

In [24]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [25]:
data = dataset.copy()
data['lower_shadow'] = data.apply(
        lambda r: min(r['Open'], r['Close']) - r['Low'], axis=1)
data['upper_shadow'] = data.apply(
        lambda r: r['High'] - max(r['Open'], r['Close']), axis=1)
data['body'] = data.apply(
        lambda r: abs(r['Close'] - r['Open']), axis=1)
data["entry_price"] = data["Open"].shift(-1)
data['color'] = data.apply(
        lambda r: 'doji' if r['Open'] == r['Close'] else ('xanh' if r['Open'] < r['Close'] else 'do' ), axis=1)

In [26]:
data

,Open,High,Low,Close,Volume,lower_shadow,upper_shadow,body,entry_price,color
Date,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,0.2,0.1,0.4,943.1,do
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,0.2,0.2,0.2,943.2,xanh
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,0.5,0.1,0.1,943.1,do
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,0.3,0.0,0.5,942.6,do
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,0.2,0.0,1.1,943.8,xanh
...,...,...,...,...,...,...,...,...,...,...
2024-06-19 14:15:00,1320.6,1325.0,1319.0,1322.7,15368,1.6,2.3,2.1,1322.6,xanh
2024-06-19 14:20:00,1322.6,1323.0,1317.0,1317.0,12067,0.0,0.4,5.6,1317.3,do
2024-06-19 14:25:00,1317.3,1318.0,1314.3,1314.5,11350,0.2,0.7,2.8,1315.1,do


In [27]:
# Only trade from 1:45PM to 2:25PM
good_time_data = data[(100*data.index.hour + data.index.minute > 1340) & (100*data.index.hour + data.index.minute < 1430)]

In [28]:
good_time_data

,Open,High,Low,Close,Volume,lower_shadow,upper_shadow,body,entry_price,color
Date,,,,,,,,,,
2018-08-13 13:45:00,948.8,948.9,947.8,948.4,1325,0.6,0.1,0.4,948.4,do
2018-08-13 13:50:00,948.4,948.6,947.4,947.7,1308,0.3,0.2,0.7,947.5,do
2018-08-13 13:55:00,947.5,950.2,947.5,949.9,2232,0.0,0.3,2.4,949.7,xanh
2018-08-13 14:00:00,949.7,949.7,948.9,949.5,1307,0.6,0.0,0.2,949.6,do
2018-08-13 14:05:00,949.6,952.0,949.6,951.9,3958,0.0,0.1,2.3,952.0,xanh
...,...,...,...,...,...,...,...,...,...,...
2024-06-19 14:05:00,1309.6,1316.7,1309.6,1316.7,14553,0.0,0.0,7.1,1316.7,xanh
2024-06-19 14:10:00,1316.7,1321.3,1314.9,1320.4,17070,1.8,0.9,3.7,1320.6,xanh
2024-06-19 14:15:00,1320.6,1325.0,1319.0,1322.7,15368,1.6,2.3,2.1,1322.6,xanh


In [29]:
long_lower_shadow = good_time_data[good_time_data.lower_shadow > 1.6]
long_lower_shadow

,Open,High,Low,Close,Volume,lower_shadow,upper_shadow,body,entry_price,color
Date,,,,,,,,,,
2018-08-15 14:20:00,951.5,952.2,949.9,951.5,5050,1.6,0.7,0.0,951.4,doji
2018-08-17 14:15:00,947.7,948.5,945.9,947.7,3060,1.8,0.8,0.0,948.1,doji
2018-08-17 14:25:00,948.6,949.4,945.0,947.0,4011,2.0,0.8,1.6,945.5,do
2018-09-12 14:00:00,958.7,958.7,955.1,957.0,4203,1.9,0.0,1.7,957.0,do
2018-10-16 14:10:00,934.6,934.6,932.9,934.5,3686,1.6,0.0,0.1,934.3,do
...,...,...,...,...,...,...,...,...,...,...
2024-06-06 14:25:00,1294.2,1295.6,1292.1,1295.6,14305,2.1,0.0,1.4,1294.9,xanh
2024-06-17 14:05:00,1308.4,1309.4,1305.3,1309.3,10349,3.1,0.1,0.9,1309.8,xanh
2024-06-17 14:25:00,1310.3,1311.1,1308.1,1309.9,6289,1.8,0.8,0.4,1309.6,do


In [30]:
long_lower_shadow['rate_1'] = long_lower_shadow.apply(
        lambda r: r['upper_shadow'] / r['lower_shadow'], axis=1)
long_lower_shadow['rate_2'] = long_lower_shadow.apply(
        lambda r: r['body'] / r['lower_shadow'], axis=1)
long_lower_shadow['rate_3'] = long_lower_shadow.apply(
        lambda r: (r['body'] + r['upper_shadow']) / r['lower_shadow'], axis=1)
long_lower_shadow['rate_3'] = long_lower_shadow['rate_3'].round(2)